## How to submit samples to EBI BioSamples

This guide explains how you can submit data to BioSamples using BioSamples API.
The guide contains Python code snippets. Some understanding of Python could be helpful to follow.

> All the services (URLs) used in this guide are from development environments to reduce test data in production servers.
> You can easily access all the production services by replacing `wwwdev` with `www` in URLs.

The guide assumes that you already have a working ENA Webin account.
If you do not have one, you can easily create a new account [here at Webin development environment](https://wwwdev.ebi.ac.uk/ena/submit/sra/#home).

> If you just created an account following the above link, it is in development environment of Webin.
All the data in development machines are replaced with production data each night.
Therefore, the account is only accessible today. Alternatively, you can create an account in
[production](https://www.ebi.ac.uk/ena/submit/sra/#home). But this will take one day to activate in development environment,
for the same reason as above.

Following are the steps we are going to follow.
1. Authenticate using ENA Webin
2. Create a sample
3. Update the sample
4. Attach a checklist to the sample for validation

Let's start by importing Python libraries and by defining variables for BioSamples and Webin URLs.

In [1]:
import requests
from datetime import datetime, timedelta

BIOSAMPLES_URL = 'https://wwwdev.ebi.ac.uk/biosamples/samples'
WEBIN_URL = 'https://wwwdev.ebi.ac.uk/ena/submit/webin/auth/token'

Now let's create two variables to hold Webin username and password. I am going to use an account we have created for demos.

In [2]:
WEBIN_USERNAME = 'Webin-59287'
WEBIN_PASSWORD = ''

Now we are ready to start the submission process.
Before the actual submission, login to the system and get a token(JWT). If you are interested, read more about JWT [here](https://jwt.io/).

In [3]:
auth_body = {
    "authRealms": ["ENA"],
    "password": WEBIN_PASSWORD,
    "username": WEBIN_USERNAME
}
response = requests.post(WEBIN_URL, json=auth_body)
if response.status_code == requests.codes.ok:
    token = response.text
else:
    raise response.raise_for_status()

In the above code snippets, first we defined the body of the authentication request.
Then we made the actual request and assigned the token to a variable called `token`

Now we have all the information to make the actual request to create a sample.
First define headers and parameters required for the requests.

In [4]:
header = {
    "Content-Type": "application/json;charset=UTF-8",
    "Accept": "application/hal+json",
    "Authorization": "Bearer " + token
}

params = {"authProvider": "WEBIN"}

Second, create the sample body. You can see sample body is in JSON format.
We have given the sample a name and assigned a release date (immediately in the past to make sample public).
Also `webinSubmissionAccountId` holds the Webin username.
Sample `characteristics` section holds the all metadata of the sample.
For BioSamples submission `organism` value is mandatory.

In [5]:
sample_request = {
    "name": "test_sample",
    "release": (datetime.now() - timedelta(hours=1)).isoformat(),
    "webinSubmissionAccountId": WEBIN_USERNAME,
    "characteristics": {
        "organism": [{
            "text": "Capsicum annuum",
            "ontologyTerms": ["http://purl.obolibrary.org/obo/NCBITaxon_4072"]
        }]
    }
}

Send request to BioSamples to create a sample.
With the request, we have included headers, parameters and sample request body.

In [1]:
submit_url = BIOSAMPLES_URL
response = requests.post(submit_url, headers=header, params=params, json=sample_request)

if response.status_code == requests.codes.created:
    sample_response = response.json()
    accession = sample_response['accession']
    print("Submitted to biosamples: " + response.text)
else:
    print("Failed to submit sample: " + sample_request["name"] + ". Reason: " + response.text)

NameError: name 'BIOSAMPLES_URL' is not defined

That's it! You have created your first sample in BioSamples.
You can read the response from the BioSamples.
Most important element of the response body is the `accession`.
This is the unique ID assigned by BioSamples to your sample.
If you want to view your samples or make any updates in the future, you will need this `accession`.

### What can we do after sample submission.
- Do you have more metadata of the sample? Update the sample by providing them.
- Does your sample compliant to a checklist, validate compliance by attaching a checklist.

Now let's try to update the previous sample.
Since we already have the sample response in `sample_response` variable,
starting from there I am going to add a bit more metadata and attach a checklist ([ERC100002](https://www.ebi.ac.uk/biosamples/schemas/certification/plant-miappe.json)) to the sample.


In [7]:
sample_request = sample_response
sample_request["characteristics"]["description"] = [{
    "text": "test sample for demo"
}]
sample_request["characteristics"]["checklist"] = [{
    "text": "ERC100002"
}]

Now the code to send the update request to BioSamples.
Important thing to notice here is that, we append the `accession` to the request URL.
We are sending a HTTP `PUT` request to update the sample.

In [8]:
update_url = BIOSAMPLES_URL + '/' + accession
response = requests.put(update_url, headers=header, params=params, json=sample_request)

response_sample = ''
if response.status_code == requests.codes.ok:
    sample_response = response.text
    print("Submitted to biosamples: " + response.text)
else:
    print("Failed to submit sample: " + sample_request["name"] + ". Reason: " + response.text)

Failed to submit sample: test_sample. Reason: Checklist validation failed: Sample validation failed: [{"dataPath":"/characteristics.biological material ID","errors":["should have required property 'biological material ID'"]}]


OHH NO! it failed. We already know why it failed.
We asked to validate against ERC100002(Plant MIAAPE) checklist, but our sample is not MIAAPE compliant.
Let's add minimum data for MIAAPE compliance and send the request.

In [9]:
sample_request = sample_response
sample_request["characteristics"]["biological material ID"] = [{
    "text": "BATEMPep1"
}]

update_url = BIOSAMPLES_URL + '/' + accession
response = requests.put(update_url, headers=header, params=params, json=sample_request)

if response.status_code == requests.codes.ok:
    response_sample = response.text
    print("Submitted to biosamples: " + response.text)
else:
    print("Failed to submit sample: " + sample_request["name"] + ". Reason: " + response.text)

Submitted to biosamples: {
  "name" : "test_sample",
  "accession" : "SAMEA7835196",
  "webinSubmissionAccountId" : "Webin-59287",
  "release" : "2021-06-11T17:56:46.312Z",
  "update" : "2021-06-11T17:56:53.482Z",
  "submitted" : "2021-06-11T17:56:46.509Z",
  "taxId" : 4072,
  "characteristics" : {
    "biological material ID" : [ {
      "text" : "BATEMPep1"
    } ],
    "checklist" : [ {
      "text" : "ERC100002"
    } ],
    "description" : [ {
      "text" : "test sample for demo"
    } ],
    "organism" : [ {
      "text" : "Capsicum annuum",
      "ontologyTerms" : [ "http://purl.obolibrary.org/obo/NCBITaxon_4072" ]
    } ]
  },
  "releaseDate" : "2021-06-11",
  "updateDate" : "2021-06-11",
  "submittedDate" : "2021-06-11",
  "submittedVia" : "JSON_API",
  "create" : "2021-06-11T17:56:46.509Z",
  "_links" : {
    "self" : {
      "href" : "https://wwwdev.ebi.ac.uk/biosamples/samples/SAMEA7835196"
    },
    "curationDomain" : {
      "href" : "https://wwwdev.ebi.ac.uk/biosamples

This time it is successful. We have just submitted a MIAAPE compliant sample to BioSamples.


This is the end of this guide. There are many other options in BioSamples API to explore. Here are few of them.
- Attach external references to the sample
- Attach sample relationships
- Reserve a large pool of BioSamples accessions

If you have any question, always feel welcome to contact us at biosamples@ebi.ac.uk
